This notebook is meant to demonstrate the transformation of an annotated notebook into a HTTP API using the Jupyter kernel gateway. The result is a simple scotch recommendation engine.

The original scotch data is from [https://www.mathstat.strath.ac.uk/outreach/nessie/nessie_whisky.html](https://www.mathstat.strath.ac.uk/outreach/nessie/nessie_whisky.html).

In [1]:
install.packages('RCurl', repos='http://cran.us.r-project.org')
library(RCurl)
library(jsonlite)


The downloaded source packages are in
	'/tmp/RtmpgGJG3Y/downloaded_packages'


Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Loading required package: bitops



The downloaded source packages are in
	'/tmp/RtmpgGJG3Y/downloaded_packages'


Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


## Data

We read the scotch data from a public Dropbox URL to make this notebook more portable. This is acceptable for small, public, demo data which is what we have here.

In [2]:
whisky_json <- getURL("https://dl.dropboxusercontent.com/u/19043899/whisky_features_df.json", ssl.verifypeer = FALSE, useragent= 'R')
whisky_similarity_json <- getURL("https://dl.dropboxusercontent.com/u/19043899/whisky_similarity_features_df.json", ssl.verifypeer = FALSE, useragent= 'R')

In [3]:
whisky_df <- fromJSON(whisky_json)
whisky_similarity_df <- fromJSON(whisky_similarity_json)
whisky_similarity_df

,Distillery,Aberfeldy,Aberlour,AnCnoc,Ardbeg,Ardmore,ArranIsleOf,Auchentoshan,Auchroisk,Aultmore,Balblair,Balmenach,Belvenie,BenNevis,Benriach,Benrinnes,Benromach,Bladnoch,BlairAthol,Bowmore,Bruichladdich,Bunnahabhain,Caol Ila,Cardhu,Clynelish,Craigallechie,Craigganmore,Dailuaine,Dalmore,Dalwhinnie,Deanston,Dufftown,Edradour,GlenDeveronMacduff,GlenElgin,GlenGarioch,GlenGrant,GlenKeith,GlenMoray,GlenOrd,GlenScotia,GlenSpey,Glenallachie,Glendronach,Glendullan,Glenfarclas,Glenfiddich,Glengoyne,Glenkinchie,Glenlivet,Glenlossie,Glenmorangie,Glenrothes,Glenturret,Highland Park,Inchgower,Isle of Jura,Knochando,Lagavulin,Laphroig,Linkwood,Loch Lomond,Longmorn,Macallan,Mannochmore,Miltonduff,Mortlach,Oban,OldFettercairn,OldPulteney,RoyalBrackla,RoyalLochnagar,Scapa,Speyburn,Speyside,Springbank,Strathisla,Strathmill,Talisker,Tamdhu,Tamnavulin,Teaninich,Tobermory,Tomatin,Tomintoul,Tormore,Tullibardine
1,Aberfeldy,1,0.9372094,0.8889566,0.5692986,0.9252892,0.8433383,0.7884045,0.9630364,0.9016696,0.8292683,0.8998201,0.8946751,0.909611,0.8447419,0.9653452,0.9756098,0.8269613,0.9756098,0.9050185,0.8946751,0.8990229,0.6747478,0.8839122,0.6265515,0.9184854,0.8589557,0.9276248,0.9210626,0.925638,0.9374253,0.8120207,0.9312684,0.8068978,0.9110136,0.8148353,0.8834522,0.9106417,0.8780488,0.9242931,0.8729435,0.814112,0.871614,0.8998201,0.9545214,0.9274495,0.7650921,0.8839122,0.8288497,0.9467531,0.8745731,0.8700222,0.89715,0.952651,0.9136499,0.8414891,0.8134394,0.9383431,0.594692,0.5426095,0.8711401,0.8711871,0.9312684,0.9212545,0.9443699,0.8155909,0.9391319,0.7804878,0.9050185,0.8536585,0.8573859,0.9622228,0.9630715,0.8068978,0.9444501,0.8395702,0.9600015,0.8215939,0.6826288,0.9370426,0.8215939,0.7499402,0.8834522,0.8780488,0.8553989,0.8121036,0.7983723
2,Aberlour,0.9372094,1,0.841435,0.5306757,0.8521543,0.837885,0.7884165,0.9488522,0.8498366,0.7780606,0.8263473,0.9559042,0.9107077,0.9186708,0.9487343,0.9548926,0.7772023,0.9195262,0.8460861,0.8534859,0.8620601,0.5940281,0.8682431,0.6114995,0.8720277,0.7925939,0.8967177,0.9015038,0.9151333,0.9515012,0.7569308,0.8950059,0.7737226,0.9058216,0.7595545,0.8540168,0.9126644,0.8487934,0.930726,0.7631953,0.7555075,0.8716273,0.855342,0.9452234,0.9062933,0.7164878,0.8889156,0.8782753,0.9478947,0.9058216,0.883086,0.8869686,0.9496885,0.841435,0.894797,0.7321067,0.9488522,0.5054959,0.43879,0.8612539,0.8513143,0.879304,0.8996469,0.8971651,0.7687061,0.922972,0.7426942,0.8806202,0.7426942,0.8633531,0.951303,0.9435642,0.8492078,0.877319,0.8056292,0.9532334,0.8190389,0.719874,0.9058216,0.7631953,0.6990621,0.8273288,0.8664766,0.8889156,0.7246573,0.8475707
3,AnCnoc,0.8889566,0.841435,1,0.4940322,0.8101915,0.7905694,0.880771,0.875,0.9433,0.7901837,0.7287987,0.7866066,0.8332051,0.855236,0.8904631,0.8889566,0.8062258,0.8395702,0.7715885,0.7866066,0.8808303,0.6099375,0.9526279,0.6099375,0.8191938,0.8959787,0.7602631,0.7926291,0.937137,0.8677218,0.8808303,0.8989845,0.6324555,0.8959787,0.6599663,0.8944272,0.8406055,0.8395702,0.8058066,0.7798129,0.7582875,0.9433,0.6883099,0.8485281,0.8164966,0.8714213,0.8948929,0.8391464,0.8444056,0.8854377,0.8514693,0.7981987,0.8198128,0.8,0.8519428,0.7099523,0.875,0.4567501,0.4648348,0.8285098,0.8436615,0.8551316,0.761387,0.8571946,0.880771,0.73951,0.716104,0.7957007,0.7407972,0.8439249,0.8413341,0.8959787,0.843274,0.9263022,0.675,0.8662831,0.8577942,0.6282809,0.9486833,0.8577942,0.6779077,0.931695,0.7407972,0.8371579,0.6640783,0.7505553
4,Ardbeg,0.5692986,0.5306757,0.4940322,1,0.6045604,0.6249068,0.2946187,0.5146169,0.5512071,0.670959,0.5500764,0.5495481,0.6860502,0.5501486,0.608016,0.6099628,0.4595786,0.6099628,0.6750232,0.6476817,0.5560359,0.9240749,0.4991518,0.9441635,0.5286727,0.716039,0.6996367,0.7294217,0.6313307,0.5581804,0.6285623,0.4874567,0.5424538,0.4773593,0.6404448,0.3682298,0.5581804,0.447306,0.6206848,0.7062965,0.4886328,0.5010974,0.6000834,0.5434077,0.5042195,0.4517699,0.4040753,0.531494,0.4885699,0.5467934,0.6769133,0.5439115,0.5161942,0.6998789,0.5845655,0.7482438,0.5146169

Drop the cluster column. Don't need it here.

In [4]:
whisky_df <- subset(whisky_df, select=-c(cluster))

## API

We need to define a global REQUEST JSON string that will be replaced on each invocation of the API. We only care about path parameters and query string arguments, so we default those to blank here for development.

In [5]:
REQUEST <- ""

Provide a way to get the names of all the scotches known by the model.

In [6]:
# GET /scotches
scotches <- subset(whisky_df, select=c(Distillery))
print(toJSON(scotches))

,Distillery
1,Aberfeldy
2,Aberlour
3,AnCnoc
4,Ardbeg
5,Ardmore
6,ArranIsleOf
7,Auchentoshan
8,Auchroisk
9,Aultmore
10,Balblair


[{"Distillery":"Aberfeldy"},{"Distillery":"Aberlour"},{"Distillery":"AnCnoc"},{"Distillery":"Ardbeg"},{"Distillery":"Ardmore"},{"Distillery":"ArranIsleOf"},{"Distillery":"Auchentoshan"},{"Distillery":"Auchroisk"},{"Distillery":"Aultmore"},{"Distillery":"Balblair"},{"Distillery":"Balmenach"},{"Distillery":"Belvenie"},{"Distillery":"BenNevis"},{"Distillery":"Benriach"},{"Distillery":"Benrinnes"},{"Distillery":"Benromach"},{"Distillery":"Bladnoch"},{"Distillery":"BlairAthol"},{"Distillery":"Bowmore"},{"Distillery":"Bruichladdich"},{"Distillery":"Bunnahabhain"},{"Distillery":"Caol Ila"},{"Distillery":"Cardhu"},{"Distillery":"Clynelish"},{"Distillery":"Craigallechie"},{"Distillery":"Craigganmore"},{"Distillery":"Dailuaine"},{"Distillery":"Dalmore"},{"Distillery":"Dalwhinnie"},{"Distillery":"Deanston"},{"Distillery":"Dufftown"},{"Distillery":"Edradour"},{"Distillery":"GlenDeveronMacduff"},{"Distillery":"GlenElgin"},{"Distillery":"GlenGarioch"},{"Distillery":"GlenGrant"},{"Distillery":"GlenKe

Let clients query for features about a specific scotch given its name.

In [7]:
# GET /scotches/:scotch
#REQUEST <- "{\"args\": {}, \"body\": \"\", \"path\": {\"scotch\": \"AnCnoc\"}}"
scotch_requested <- fromJSON(REQUEST)$path$scotch
if(is.null(scotch_requested)) {
    scotch_requested <- "Talisker"
}
scotch_features <- whisky_df[whisky_df$Distillery==scotch_requested,]
print(toJSON(scotch_features))

,Distillery,Body,Sweetness,Smoky,Medicinal,Tobacco,Honey,Spicy,Winey,Nutty,Malty,Fruity
3,AnCnoc,1,3,2,0,0,2,0,0,2,2,3


[{"Distillery":"AnCnoc","Body":1,"Sweetness":3,"Smoky":2,"Medicinal":0,"Tobacco":0,"Honey":2,"Spicy":0,"Winey":0,"Nutty":2,"Malty":2,"Fruity":3}] 


Let clients request a set of scotches similar to the one named. Let clients specify how many results they wish to receive (count) and if they want all of the raw feature data included in the result or not (include_features).

In [8]:
# GET /scotches/:scotch/similar
#REQUEST <- "{\"args\": {\"count\": 4, \"include_features\": \"True\"}, \"body\": \"\", \"path\": {\"scotch\": \"AnCnoc\"}}"
scotch_requested <- fromJSON(REQUEST)$path$scotch
if(is.null(scotch_requested)) {
    scotch_requested <- "Talisker"
}
similarity_count <- fromJSON(REQUEST)$args$count
if(is.null(similarity_count)) {
    similarity_count <- 5
}
features_requested <- fromJSON(REQUEST)$args$include_features
if(is.null(features_requested)) {
    features_requested <- ""
}

#subset df columns to Distillery and the scotch requested
top_similar <- subset(whisky_similarity_df, select=c('Distillery',scotch_requested))
#order df by scotch requested
top_similar <- top_similar[order(-top_similar[scotch_requested]),]
#take the top similarity count rows of the df
top_similar <- top_similar[2:(similarity_count+1),]

if (features_requested == "True") {
    top_similar_with_features <- merge(x = top_similar, y = whisky_df, by = "Distillery", all.x = TRUE)
    #print(top_similar_with_features)
    print(toJSON(top_similar_with_features))
} else {
    #print(top_similar)
    print(toJSON(top_similar))
}

    Distillery    AnCnoc Body Sweetness Smoky Medicinal Tobacco Honey Spicy
1     Aultmore 0.9433000    2         2     1         0       0     1     0
2       Cardhu 0.9526279    1         3     1         0       0     1     1
3 Glenallachie 0.9433000    1         3     1         0       0     1     1
4       Tamdhu 0.9486833    1         2     1         0       0     2     0
  Winey Nutty Malty Fruity
1     0     2     2      2
2     0     2     2      2
3     0     1     2      2
4     1     1     2      2
[{"Distillery":"Aultmore","AnCnoc":0.9433,"Body":2,"Sweetness":2,"Smoky":1,"Medicinal":0,"Tobacco":0,"Honey":1,"Spicy":0,"Winey":0,"Nutty":2,"Malty":2,"Fruity":2},{"Distillery":"Cardhu","AnCnoc":0.9526,"Body":1,"Sweetness":3,"Smoky":1,"Medicinal":0,"Tobacco":0,"Honey":1,"Spicy":1,"Winey":0,"Nutty":2,"Malty":2,"Fruity":2},{"Distillery":"Glenallachie","AnCnoc":0.9433,"Body":1,"Sweetness":3,"Smoky":1,"Medicinal":0,"Tobacco":0,"Honey":1,"Spicy":1,"Winey":0,"Nutty":1,"Malty":2,"Fruity"